# Diabetes Classification using Classical Machine Learning Algorithms

## Comparision on Support Vector Machine, Logistic Regression, Random Forest, Decision Tree, KNN

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score


In [2]:
df = pd.read_csv('data/diabetes.csv')

X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
models = {
    'SVM': SVC(kernel='rbf', random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'KNN': KNeighborsClassifier()
}

In [6]:
for name, model in models.items():
    print(f"{name}:")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("\tAccuracy:", accuracy_score(y_test, y_pred))
    print("\tConfusion Matrix:", end=" ")
    print(confusion_matrix(y_test, y_pred))

SVM:
	Accuracy: 0.7662337662337663
	Confusion Matrix: [[87 12]
 [24 31]]
Logistic Regression:
	Accuracy: 0.7467532467532467
	Confusion Matrix: [[78 21]
 [18 37]]
Random Forest:
	Accuracy: 0.7207792207792207
	Confusion Matrix: [[77 22]
 [21 34]]
Decision Tree:
	Accuracy: 0.7467532467532467
	Confusion Matrix: [[75 24]
 [15 40]]
KNN:
	Accuracy: 0.6623376623376623
	Confusion Matrix: [[70 29]
 [23 32]]


/Users/measaverb/miniconda3/envs/ai-class/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
